In [46]:
import pathlib
import pickle
import requests
import zipfile
import io

import pandas as pd

In [47]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
print(DATA_DIR)

c:\Users\Marlon\Desktop\6_Semestre\ML\Projeto2_MachineLearning_23.2\data


In [48]:
DATA_DIR.mkdir(parents=True, exist_ok=True)

In [49]:
raw_data_dir = DATA_DIR / 'raw'
raw_data_dir.mkdir(parents=True, exist_ok=True)
print(raw_data_dir)

c:\Users\Marlon\Desktop\6_Semestre\ML\Projeto2_MachineLearning_23.2\data\raw


In [50]:
raw_data_file_path = DATA_DIR / 'raw' / 'glaucoma_dataset.csv'
print(raw_data_file_path)

c:\Users\Marlon\Desktop\6_Semestre\ML\Projeto2_MachineLearning_23.2\data\raw\glaucoma_dataset.csv


In [51]:
filesize = raw_data_file_path.stat().st_size
print(f'This file has {filesize} bytes')

This file has 3380533 bytes


In [52]:
raw_data = pd.read_csv(raw_data_file_path)

In [53]:
raw_data.shape

(10000, 17)

In [54]:
raw_data.head()

,Patient ID,Age,Gender,Visual Acuity Measurements,Intraocular Pressure (IOP),Cup-to-Disc Ratio (CDR),Family History,Medical History,Medication Usage,Visual Field Test Results,Optical Coherence Tomography (OCT) Results,Pachymetry,Cataract Status,Angle Closure Status,Visual Symptoms,Diagnosis,Glaucoma Type
0,62431,69,Male,LogMAR 0.1,19.46,0.42,No,Diabetes,"Amoxicillin, Lisinopril, Omeprazole, Atorvasta...","Sensitivity: 0.54, Specificity: 0.75","RNFL Thickness: 86.48 µm, GCC Thickness: 64.14...",541.51,Present,Open,"Tunnel vision, Eye pain, Nausea",No Glaucoma,Primary Open-Angle Glaucoma
1,68125,69,Female,LogMAR 0.1,18.39,0.72,No,Hypertension,"Lisinopril, Amoxicillin, Atorvastatin, Ibuprof...","Sensitivity: 0.72, Specificity: 0.88","RNFL Thickness: 96.88 µm, GCC Thickness: 56.48...",552.77,Absent,Open,"Redness in the eye, Vision loss, Tunnel vision",No Glaucoma,Juvenile Glaucoma
2,63329,67,Female,20/40,23.65,0.72,No,Hypertension,"Amoxicillin, Ibuprofen, Metformin, Atorvastati...","Sensitivity: 0.56, Specificity: 0.8","RNFL Thickness: 89.81 µm, GCC Thickness: 59.05...",573.65,Absent,Closed,"Halos around lights, Vision loss, Redness in t...",No Glaucoma,Juvenile Glaucoma
3,47174,23,Male,LogMAR 0.0,18.04,0.61,No,NaN,"Ibuprofen, Aspirin","Sensitivity: 0.6, Specificity: 0.93","RNFL Thickness: 87.25 µm, GCC Thickness: 63.98...",590.67,Absent,Closed,"Nausea, Nausea, Halos around lights",No Glaucoma,Congenital Glaucoma
4,67361,21,Male,LogMAR 0.1,15.87,0.30,No,Diabetes,"Amoxicillin, Omeprazole, Aspirin, Ibuprofen, A...","Sensitivity: 0.82, Specificity: 0.9","RNFL Thickness: 82.61 µm, GCC Thickness: 66.01...",588.41,Absent,Closed,"Eye pain, Eye pain, Tunnel vision",No Glaucoma,Primary Open-Angle Glaucoma


In [55]:
data = raw_data.copy()

In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Patient ID                                  10000 non-null  int64  
 1   Age                                         10000 non-null  int64  
 2   Gender                                      10000 non-null  object 
 3   Visual Acuity Measurements                  10000 non-null  object 
 4   Intraocular Pressure (IOP)                  10000 non-null  float64
 5   Cup-to-Disc Ratio (CDR)                     10000 non-null  float64
 6   Family History                              10000 non-null  object 
 7   Medical History                             7453 non-null   object 
 8   Medication Usage                            8769 non-null   object 
 9   Visual Field Test Results                   10000 non-null  object 
 10  Optical Coh

In [57]:
data.dtypes.value_counts()

object     12
float64     3
int64       2
Name: count, dtype: int64

In [58]:
medications = set()
for item in data['Medication Usage']:
    try:
        medications.update(item.split(', '))
    except AttributeError:
        pass


In [59]:
medications

{'Amoxicillin',
 'Aspirin',
 'Atorvastatin',
 'Ibuprofen',
 'Lisinopril',
 'Metformin',
 'Omeprazole'}

In [60]:
medical_history = set()
for item in data['Medical History']:
    try:
        medical_history.update(item.split(', '))
    except AttributeError:
        pass

In [61]:
medical_history

{'Diabetes', 'Glaucoma in family', 'Hypertension'}

Trocando dados faltantes por None

In [62]:
data['Medication Usage'].fillna('None', inplace=True)
data['Medical History'].fillna('None', inplace=True)

In [63]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Patient ID                                  10000 non-null  int64  
 1   Age                                         10000 non-null  int64  
 2   Gender                                      10000 non-null  object 
 3   Visual Acuity Measurements                  10000 non-null  object 
 4   Intraocular Pressure (IOP)                  10000 non-null  float64
 5   Cup-to-Disc Ratio (CDR)                     10000 non-null  float64
 6   Family History                              10000 non-null  object 
 7   Medical History                             10000 non-null  object 
 8   Medication Usage                            10000 non-null  object 
 9   Visual Field Test Results                   10000 non-null  object 
 10  Optical Coh

Separando a feature "Medication Usage" em features booleanas

In [64]:
for item in medications:
    data[item] = data['Medication Usage'].str.contains(item).astype(int)

data = data.drop('Medication Usage', axis=1)

In [65]:
visual_symptoms = set()
for item in data['Visual Symptoms']:
    try:
        visual_symptoms.update(item.split(', '))
    except AttributeError:
        pass

In [66]:
visual_symptoms

{'Blurred vision',
 'Eye pain',
 'Halos around lights',
 'Nausea',
 'Redness in the eye',
 'Tunnel vision',
 'Vision loss',
 'Vomiting'}

In [67]:
for item in medications:
    data[item] = data['Visual Symptoms'].str.contains(item).astype(int)

data = data.drop('Visual Symptoms', axis=1)

In [68]:
data['Glaucoma Type'].value_counts()

Glaucoma Type
Juvenile Glaucoma              1734
Normal-Tension Glaucoma        1699
Primary Open-Angle Glaucoma    1668
Secondary Glaucoma             1657
Congenital Glaucoma            1641
Angle-Closure Glaucoma         1601
Name: count, dtype: int64

Separando a coluna "Visual Field Test Results" em "VFTR Sensitivity" e "VFTR Specificity"

In [78]:
#Separa a coluna 'Visual Field Test Results' em "VFTR Sensitivity" e "VFTR Specificity"
data[['VFTR Sensitivity', 'VFTR Specificity']] = data['Visual Field Test Results'].str.split(',', expand=True)
data['VFTR Sensitivity'] = data['VFTR Sensitivity'].str.replace('Sensitivity: ', '').astype(float)
data['VFTR Specificity'] = data['VFTR Specificity'].str.replace('Specificity: ', '').astype(float)
data = data.drop('Visual Field Test Results', axis=1)

In [82]:
for item in data["Optical Coherence Tomography (OCT) Results"]:
    print(item)
    break

RNFL Thickness: 86.48 µm, GCC Thickness: 64.14 µm, Retinal Volume: 5.63 mm³, Macular Thickness: 283.67 µm


Separando a coluna "Optical Coherence Tomography (OCT) Results" em "RNFL Thickness", "GCC Thickness", "Retinal Volume" e "Macular Thickness"

In [97]:
data[['RNFL Thickness', 'GCC Thickness', 'Retinal Volume', 'Macular Thickness']] = data['Optical Coherence Tomography (OCT) Results'].str.split(',', expand=True)

data['RNFL Thickness'] = data['RNFL Thickness'].str.replace('RNFL Thickness: ', '')
data['RNFL Thickness'] = data['RNFL Thickness'].str.replace('µm', '').astype(float)
data['GCC Thickness'] = data['GCC Thickness'].str.replace('GCC Thickness: ', '')
data['GCC Thickness'] = data['GCC Thickness'].str.replace('µm', '').astype(float)
data['Retinal Volume'] = data['Retinal Volume'].str.replace('Retinal Volume: ', '')
data['Retinal Volume'] = data['Retinal Volume'].str.replace('mm³', '').astype(float)
data['Macular Thickness'] = data['Macular Thickness'].str.replace('Macular Thickness: ', '')
data['Macular Thickness'] = data['Macular Thickness'].str.replace('µm', '').astype(float)

data = data.drop('Optical Coherence Tomography (OCT) Results', axis=1)

In [98]:
data.head()

,Patient ID,Age,Gender,Visual Acuity Measurements,Intraocular Pressure (IOP),Cup-to-Disc Ratio (CDR),Family History,Medical History,Pachymetry,Cataract Status,...,Metformin,Lisinopril,Amoxicillin,Atorvastatin,VFTR Sensitivity,VFTR Specificity,RNFL Thickness,GCC Thickness,Retinal Volume,Macular Thickness
0,62431,69,Male,LogMAR 0.1,19.46,0.42,No,Diabetes,541.51,Present,...,0,0,0,0,0.54,0.75,86.48,64.14,5.63,283.67
1,68125,69,Female,LogMAR 0.1,18.39,0.72,No,Hypertension,552.77,Absent,...,0,0,0,0,0.72,0.88,96.88,56.48,5.69,261.48
2,63329,67,Female,20/40,23.65,0.72,No,Hypertension,573.65,Absent,...,0,0,0,0,0.56,0.80,89.81,59.05,5.96,282.34
3,47174,23,Male,LogMAR 0.0,18.04,0.61,No,None,590.67,Absent,...,0,0,0,0,0.60,0.93,87.25,63.98,6.44,262.86
4,67361,21,Male,LogMAR 0.1,15.87,0.30,No,Diabetes,588.41,Absent,...,0,0,0,0,0.82,0.90,82.61,66.01,6.16,261.78
